In [5]:
import numpy as np 
import IPython.display as ipd
import scipy.io.wavfile as wav
import ipywidgets as widgets
import pyaudio
from scipy import signal
import sys
sys.path.append("./..")


grabacion_mic=[]

def sonido(x,fs,m='none'):
    """Abre un interfaz para la reproducción de auido
    Argumentos de Entrada:
        x (np.ndarray): variable con la forma de onda de la señal a reproducir
        fs (float): frecuencia de muestreo de la señal a reproducir
        m (variable): Por defecto vacía, y el audio se reproducirá normalizado respecto su máximo valor absoluto. En otro caso, indica el reproductor que normalice el audio respecto del formato int16
    """
    if m!='none':
        x=x.astype(np.float64)
        x=x/2**15
        ipd.display(ipd.Audio(x.T,rate=fs,normalize=False))
    else:
        ipd.display(ipd.Audio(x.T,rate=fs))
    
def lee_audio(fichero):
    """Lee fichero de audio wav (devuelve frecuencia de muestreo y array)
    Argumentos de Entrada:
        fichero (String): Cadena con la rura y el archivo de audio en formato wav
        
    Salida:
        fs (float): frecuencia de muestreo del audio
        x (np.ndarray): variable con las muestras de la señal de audio
    """    
    fs,x = wav.read(fichero)
    return fs,x

def pitchshifting_tr(fs,Tb,Tdg=0.03,x=np.array([None,None])):
    '''Realiza en tiempo real el efecto de pitch shifting.

      Argumentos de entrada:
        fs (escalar): frecuencia de muestreo
        Tb (escalar): Duración del bloque de datos
        Td (escalar): Máximo retardo retardo. Por defecto 30ms 
        x: Señal a representar o NONE para analizar la señal captada por el micrófono'''
    
    
    global In,fss,CHUNK,xss,Nbuf,m,Td,n,numtramas,Nb,ph1,ph2,Overlap,fgain,sd,Ret1,Ret2
    
    Ret1=0
    Ret2=0
    B=int(np.ceil(fs*Tb))
    Td=1*Tdg
    Overlap=0.1  # Porcentaje de solapamiemto entre las dos líneas de retardo (10%)
    fgain = 1 / Overlap
    Nb=int(np.ceil(fs*Tb))  # Muestras del bloque
    sd=int(np.ceil(fs*Td))  # Muestras del retardo 

    Nbuf=np.zeros(2+Nb+sd)  # Buffer de datos incluyendo líneas de retardo

    CHUNK=1*B 
    
    ph1 = 0                 # fases normalizadas de las funciones de las líneas de retardo
    ph2 = (1 - Overlap)


    In=False
   
    if x[0]==None:
        In=True
    else:
        numtramas=np.ceil(len(x)/CHUNK)
        xss=np.zeros(int(numtramas)*CHUNK)
        xss[0:len(x)]=x

  
    n=0

                 
    
    fss=1*fs

    # Botones y controles
    botonDet=widgets.Button(
            description='Detener',
            disabled=True,
            button_style='', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Description',
            )
    botonEmp=widgets.Button(
            description='Empezar',
            disabled=False,
            button_style='', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Description',
            )
    
    barraFactor=widgets.FloatSlider(
            style = {'description_width': 'auto'},
            value=0,
            min=-12,
            max=12,
            step=0.5,
            description='Factor de desplazaminto del Pitch',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.1f',
            layout=widgets.Layout(width='70%', height='100px'),
            )

        # MOSAICO VISUALIZACIÓN
    display(widgets.HBox([botonEmp, botonDet, barraFactor]))
    
    # Instancia a clase pyaudio
    p = pyaudio.PyAudio()
    
    
    def callback(in_data, frame_count, time_info, status):
        global In,fss,CHUNK,xss,Nbuf,m,Td,n,numtramas,Nb,ph1,ph2,Overlap,fgain,sd,Ret1,Ret2
        flag= pyaudio.paContinue
        audio_data = np.frombuffer(in_data, dtype=np.int16)
        yb=np.zeros(CHUNK)
        m = (1-2**(barraFactor.value/12)) # Variación del retardo (en muestras) de una muesra a la siguiente
        pRate = m / Td          # Variación del retardo en muestras por segundo
        pstep = pRate / fss      # Paso de las fases (adimensional)

        if In==False:
            if n==numtramas*CHUNK:
                n=0
            in_data=xss[n:n+CHUNK]
            n=n+CHUNK
        else:
            in_data=audio_data
        
         
        Nbuf=np.concatenate((in_data[-1::-1],Nbuf[:-Nb]))
        if m==0:
            out_data=1*in_data
            
        else:
            # Procesamos muestra a muestra dentro de cada bloque
            
            for muestra in range(Nb):
                ph1 = (ph1 + pstep)%1
                ph2 = (ph2 + pstep)%1
                
            # línea de retardo 2 se acerca a su fin. Comenzamos solape con linea de retardo 1
                if ((ph1 < Overlap) and (ph2 >= (1 - Overlap))):
                    Ret1 = sd * ph1 
                    Ret2 = sd * ph2 
                    
                    Gan1  = np.cos((1 - (ph1* fgain)) * np.pi/2)
                    Gan2  = np.cos(((ph2 - (1 - Overlap)) * fgain) * np.pi/2)
                    
            # Línea de retardo 1 está activa
                elif ((ph1 > Overlap) and (ph1 < (1 - Overlap))):
                    # El retardo de la línea 2 se matiene fijo mientras la línea 1 está activa
                    ph2 = 0.0
                    Ret1 = sd * ph1 
             
                    Gan1 = 1.0
                    Gan2 = 0.0
                   
            # Linea de retardo 1 se acerca a su fin. Comenzamos solape con linea de retardo 2
                elif ((ph1 >= (1 - Overlap)) and (ph2 < Overlap)):
                    Ret1 = sd * ph1
                    Ret2 = sd * ph2 
                    
                    Gan1 = np.cos(((ph1 - (1 - Overlap)) * fgain) * np.pi/2)
                    Gan2 = np.cos((1 - (ph2* fgain)) * np.pi/2)
                    
            # Linea de retardo 2 está activa
                elif((ph2 > Overlap) and (ph2 < (1 - Overlap))):
                    # El retardo de la línea 1 se matiene fijo mientras la línea 2 está activa
                    ph1 = 0.0    
                    Ret2 = sd * ph2 
                    
                    Gan1 = 0.0
                    Gan2 = 1.0
                    
            
            # Aplicamos las líneas de retardo
                
            #Linea 1
                  
                retardo1 = Nb-muestra + Ret1  
                Nretardo1=np.floor(retardo1)      # Redondeo del retardo a nº de muestras enteras.
                frac1=retardo1-Nretardo1           # Calculamos el error al redondear para interpolar (en el caso en el que el retardo a aplicar no sea un númeto de muestras entero, interpolamos un valor entre las dos muestras más cercanas).
                y1=Nbuf[int(Nretardo1)]*(1-frac1)+Nbuf[int(Nretardo1)+1]*(frac1) 
                
            #Linea 2
                retardo2 = Nb-muestra + Ret2
                Nretardo2=np.floor(retardo2)      # Redondeo del retardo a nº de muestras enteras.
                frac2=retardo2-Nretardo2           # Calculamos el error al redondear para interpolar (en el caso en el que el retardo a aplicar no sea un númeto de muestras entero, interpolamos un valor entre las dos muestras más cercanas).
                y2=Nbuf[int(Nretardo2)]*(1-frac2)+Nbuf[int(Nretardo2)+1]*(frac2) 
            
            # Solapamos ambas líneas
                yb[muestra]=Gan1*y1+Gan2*y2

        
        
            out_data=1*yb
           
        return (bytes(out_data.astype(np.int16)),flag)
        
    stream = p.open(format=pyaudio.paInt16,
                channels=1,
                rate=fss,
                input=True,
                output=True,
                start=False,
                frames_per_buffer=CHUNK,
                stream_callback=callback)    
    
    def on_button_clickedDet(b):
        stream.stop_stream()
        stream.close()
        p.terminate()
        botonDet.disabled=True

    def on_button_clickedEmp(b):
        stream.start_stream()
        botonDet.disabled=False
        botonEmp.disabled=True       
        

        
    botonDet.on_click(on_button_clickedDet)
    botonEmp.on_click(on_button_clickedEmp)

            

fs,x=lee_audio('/Audio/melodia_solista.wav') # Puedes cargar aqui el audio que prefieras, audio de prueba por defecto.



### Si quieres cambiar el pitch de tu voz descomenta la linea de fs (asi aseguramos que funcione en tarjetas de sonido menos potentes) y borra 
### la variable x en la funcion pitchshifting_tr de abajo. Si quieres cambiar el pitch de una canción dejalo como está.

# fs=16000

pitchshifting_tr(fs,0.03,x=x) # Aqui puedes cambiar el pitch de tu canción
